In [ ]:
using StatProfilerHTML
using Profile

# Clear any old profiling data
Profile.clear()

function test()
    for i in 1:100
        substitute(A, xi, dA_dt)
    end
end
# Run profiling
test()
@profile test() 

# Create HTML report
statprofilehtml()

In [ ]:
##### Sorting and Simplifying IndexeProduct and IndexedCumulant
# Sorting the elements
function sort_indices!(ip::IndexedProduct)
    # sort each inner vector
    for v in ip.indices
        sort!(v)
    end
    # then sort the vector of vectors lex order
    sort!(ip.indices)
    return ip
end
import Base: isless, <
function isless(a::IndexedProduct, b::IndexedProduct)
    # ensure each is normalized before comparing
    sort_indices!(a)
    sort_indices!(b)
    return isless(a.indices, b.indices)
end
# Test 
ip1 = IndexedProduct(1, [[5,2,3], [4,1,6]])
println(sort_indices!(ip1))

In [ ]:
# Simplify by adding equal terms and removing zero terms 
function simplify_(products::Vector{IndexedProduct})
    # 1) normalize each product’s indices (in‑place sort of the inner vectors)
    for ip in products
        sort_indices!(ip)
    end

    # 2) sort the whole list
    sort!(products)

    # 3) merge
    result = IndexedProduct[]
    for ip in products
        if !isempty(result) && result[end].indices == ip.indices
            # build a new IndexedProduct with the summed coeff
            new_coeff = result[end].coeff + ip.coeff
            # keep the same sorted indices vector
            result[end] = IndexedProduct(new_coeff, result[end].indices)
        else
            push!(result, IndexedProduct(ip.coeff, copy.(ip.indices)))
        end
    end

    # 4) drop zeros
    filter!(ip -> ip.coeff != 0, result)
    return result
end
# Test 
data = [IndexedProduct(2, [[3,1], [2]]), IndexedProduct(5, [[1,3], [2]]), IndexedProduct(-6, [[3], [1,2]])]
println(simplify_(data))